## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Scraping Libraries
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
#from selenium.webdriver.chrome.options import Options
from time import sleep
import re
from IPython.core.display import display, HTML

In [3]:
def open_browser(alt_user_name = 'Thank you for your website', path = '../Garage/chromedriver'):
    opts = webdriver.chrome.options.Options()
    opts.add_argument("user-agent=" + str(alt_user_name))
             # Path to Chromedriver
    return webdriver.Chrome(executable_path = path)#, options=opts)

In [4]:
def html_table_to_df(soup):
    proto_df = []
    header = "Unsaved"
    for row in soup.find_all('tr'):
        current_row = {}
        maybe_header = [word.text.strip('\n').strip(' ') for word in row.find_all('th')]
        maybe_row    = [word for word in row.find_all('td')]
        if len(maybe_header) > 0:
            header = maybe_header
        elif len(maybe_row) > 0:

            for col_num in range(len(header)):
                current_row[header[col_num]] = maybe_row[col_num].text.strip('\n')

            proto_df.append(current_row)
        else:
            print("Something might have gone wrong")
            print(row)
    return pd.DataFrame(proto_df)

# DnD Subclass Scraping
There was a recent update on the 5e Wiki in which they deleted many listings of subclasses.  They likely did this for monitary reasions.  I personally like to still have this information such taht I'll be able to learn a bit about which subclass.  If I ever do get the chance to play 5e D&D then I plan to use and pay for the appropriate content from DnDBeyond.
To scrape the information I found the url from onw of the pages on the wayback machine and scraped the full HTML from each.

### Gathering the Data

In [85]:
browser_2 = open_browser()
browser_2.get('https://web.archive.org/web/20190207025034/https://dnd5e.fandom.com/wiki/monk')

In [86]:
# Copied the main class names by copy/paste to save some time and energy
class_names = ['Barbarian',
'Bard',
'Cleric',
'Druid',
'Fighter',
'Monk',
'Paladin',
'Ranger',
'Rogue',
'Sorcerer',
'Warlock',
'Wizard']

In [89]:
# Running through each class and saving the html of each page
archived_htmls = []
for class_name in class_names:
    browser_2.get('https://web.archive.org/web/20190207025034/https://dnd5e.fandom.com/wiki/' + class_name)
    archived_htmls.append(browser_2.page_source)

In [91]:
# Saving in csv just in case.
to_save = pd.DataFrame(archived_htmls)

to_save.to_csv('./data/DnD_5e_class_htmls.csv')

In [142]:
test_soup#.find_all(attrs = {'class' : 'mw-headline'})

<!DOCTYPE html>
<html class="macos desktop landscape" dir="ltr" lang="en" xmlns="http://www.w3.org/1999/xhtml"><head><script async="" data-jsonpid="" src="https://cdn-gl.imrworldwide.com/novms/js/2/nlsSDK600.bundle.min.js"></script><script async="" src="https://web.archive.org/web/20190207024822/https://cdn-gl.imrworldwide.com/conf/P26086A07-C7FB-4124-A679-8AC404198BA7.js#name=nlsnInstance&amp;ns=NOLBUNDLE"></script><script async="" src="https://web.archive.org/web/20190207024822/https://www.google-analytics.com/analytics.js"></script><script src="//archive.org/includes/analytics.js?v=cf34f82" type="text/javascript"></script>
<script type="text/javascript">window.addEventListener('DOMContentLoaded',function(){var v=archive_analytics.values;v.service='wb';v.server_name='wwwb-app100.us.archive.org';v.server_ms=148;archive_analytics.send_pageview({});});</script>
<script charset="utf-8" src="/static/js/ait-client-rewrite.js?v=1549496156" type="text/javascript"></script>
<script type="text

In [130]:
soup_list = [BeautifulSoup(html) for html in archived_htmls]

### Getting the data to view in an easy way

In [207]:
# Had to look this bit up, but was stoked that I already have IPython installed :)
from IPython.core.display import display, HTML
display(HTML('<h3>It works! :D</h3>'))

In [163]:
count = 0
class_pages = {}
for item in soup_list:
    class_pages[item.find(attrs = {'class':
        'page-header__title'}).text] = item.find_all(attrs = {'id':'mw-content-text'})
    print(count)
    count += 1
    

0
1
2
3
4
5
6
7
8
9
10
11


In [164]:
type(item)

bs4.BeautifulSoup

### Saving the Data 

In [170]:
file = open('./data/testfile.txt','w') 

file.write("this is a test.  If we see this then it worked :)")
file.close()

In [186]:
for page in class_pages.keys():
    file = open('./data/5e_classes/'+page+'.txt','w') 
    file.write(str(class_pages[page][0]))
    file.close()

In [185]:
for page in class_pages.keys():
    print(len(class_pages[page][0]))

244
249
673
290
403
319
385
288
294
415
634
540


In [194]:
# Cool, this works and as such all will be fine
test_class = open('./data/5e_classes/Bard.html').read()

display(HTML(test_class))

Level,Proficiency Bonus,Features,Cantrips Known,Spells Known,1st,2nd,3rd,4th,5th,6th,7th,8th,9th
1st,+2,"Spellcasting, Bardic Inspiration (d6)",2,4,2,—,—,—,—,—,—,—,—
2nd,+2,"Jack of All Trades, Song of Rest (d6)",2,5,3,—,—,—,—,—,—,—,—
3rd,+2,"Bard College, Expertise",2,6,4,2,—,—,—,—,—,—,—
4th,+2,Ability Score Improvement,3,7,4,3,—,—,—,—,—,—,—
5th,+3,"Bardic Inspiration (d8), Font of Inspiration",3,8,4,3,2,—,—,—,—,—,—
6th,+3,"Countercharm, Bard College feature",3,9,4,3,3,—,—,—,—,—,—
7th,+3,—,3,10,4,3,3,1,—,—,—,—,—
8th,+3,Ability Score Improvement,3,11,4,3,3,2,—,—,—,—,—
9th,+4,Song of Rest (d8),3,12,4,3,3,3,1,—,—,—,—
10th,+4,"Bardic Inspiration (d10), Expertise, Magical Secrets",4,14,4,3,3,3,2,—,—,—,—


# Spells
Scraping all the information on the 5e Spells for the same reason!

Reference Codes:
 - PHB : Player's Hand Book
 - XGE : Xanathar's Guide to Everything
 - SCAG : Swords Coast Adventure's Guide
 - EEPC : Elemental Evil Player's Companion
 - GGR : Guildmasters' Guide to Ravnica
 - Unknown: UARA, UASS, UAMM, UATOBM

In [4]:
spell_window = open_browser()

In [6]:
spell_window.get('https://web.archive.org/web/20190207025036/https://dnd5e.fandom.com/wiki/List_of_Spells')

In [7]:
spell_soup = BeautifulSoup(spell_window.page_source)

In [25]:
tables = spell_soup.find_all('table')
tables.pop(0)
tables.pop(0)
len(tables)

17

In [151]:
proto_df = []
for current_table in tables:
    header = "Unsaved"
    for row in current_table.find_all('tr'):
        #print(len(row))
        current_row = {}
        maybe_header = [word.text.strip('\n').strip(' ') for word in row.find_all('th')]
        maybe_row    = [word for word in row.find_all('td')]
        if len(maybe_header) > 0:
            header = maybe_header
        elif len(maybe_row) > 0:
            
            for col_num in range(len(header)):
                current_row[header[col_num]] = maybe_row[col_num].text.strip('\n')
                    
            proto_df.append(current_row)
            
        else:
            print("Something might have gone wrong")
            print(row)

In [ ]:
df_base = pd.DataFrame(proto_df)

In [96]:
link_list = []
for table in tables:
    for element in table.find_all('a'):
        row = {
            'href'  : element.attrs['href'],
            'title' : element.attrs['title']}
        link_list.append(row)

In [132]:
df_links = pd.DataFrame(link_list, columns = ['href', 'title', 'html'])

In [133]:
df_links['href'] = df_links['href'].map(lambda x: 'https://web.archive.org' + x)

In [136]:
df_links.head()

,href,title,html
0,https://web.archive.org/web/20190207025036/htt...,Acid Splash,NaN
1,https://web.archive.org/web/20190207025036/htt...,Blade Ward,NaN
2,https://web.archive.org/web/20190207025036/htt...,Booming Blade,NaN
3,https://web.archive.org/web/20190207025036/htt...,Chill Touch,NaN
4,https://web.archive.org/web/20190207025036/htt...,Control Flames,NaN


In [146]:
for row_num in range(len(link_list)):#df_links.shape[0]):
    spell_window.get('https://web.archive.org' + link_list[row_num]['href'])
    spell_soup = BeautifulSoup(spell_window.page_source)
    spell_html = spell_soup.find(attrs = {'id' : 'mw-content-text'})
    link_list[row_num]['html'] = spell_html

In [198]:
pd.DataFrame(link_list).to_csv('./data/a_mess_on_April_6.csv', index = False)

In [142]:
df_links

,href,title,html
0,https://web.archive.org/web/20190207025036/htt...,Acid Splash,test
1,https://web.archive.org/web/20190207025036/htt...,Blade Ward,NaN
2,https://web.archive.org/web/20190207025036/htt...,Booming Blade,NaN
3,https://web.archive.org/web/20190207025036/htt...,Chill Touch,NaN
4,https://web.archive.org/web/20190207025036/htt...,Control Flames,NaN
5,https://web.archive.org/web/20190207025036/htt...,Create Bonfire,NaN
6,https://web.archive.org/web/20190207025036/htt...,Dancing Lights,NaN
7,https://web.archive.org/web/20190207025036/htt...,Druidcraft,NaN
8,https://web.archive.org/web/20190207025036/htt...,Eldritch Blast,NaN
9,https://web.archive.org/web/20190207025036/htt...,Fire Bolt,NaN


In [118]:

spell_window.get(test_href)

In [164]:
def html_table_to_df(soup):
    proto_df = []
    header = "Unsaved"
    for row in soup.find_all('tr'):
        current_row = {}
        maybe_header = [word.text.strip('\n').strip(' ') for word in row.find_all('th')]
        maybe_row    = [word for word in row.find_all('td')]
        if len(maybe_header) > 0:
            header = maybe_header
        elif len(maybe_row) > 0:

            for col_num in range(len(header)):
                current_row[header[col_num]] = maybe_row[col_num].text.strip('\n')

            proto_df.append(current_row)
        else:
            print("Something might have gone wrong")
            print(row)
    return pd.DataFrame(proto_df)

In [184]:
num_link = 100
list_table_html = link_list[num_link]['html'].find('table').find_all('tr')
for num_row in list_table_html:
    value_column = list_table_html[num_row].find('th')
    value_cell   = list_table_html[num_row].find('td')
    link_list[num_link][value_column] = value_cell

TypeError: list indices must be integers or slices, not Tag

In [191]:
list_tableless = []
for num_link in range(len(link_list)):
    try:
        list_table_html = link_list[num_link]['html'].find('table').find_all('tr')
        for num_row in range(len(list_table_html)):
            value_column = list_table_html[num_row].find('th')
            value_cell   = list_table_html[num_row].find('td')
            link_list[num_link][value_column] = value_cell
    except:
        list_tableless.append(num_link)

In [190]:
for num_link in range(len(link_list)):
    try:
        potential_caption = link_list[num_link]['html'].find('caption')
        link_list[num_link]['caption'] = potential_caption
    except:
        print("skipped", num_link)

In [188]:
list_links = link_list

In [100]:
df_base = pd.DataFrame(proto_df)

In [161]:
df_base.columns
# Range, Components, Duration, Discription, Level

Index(['Casting Time', 'Conc.', 'Concentration', 'Name', 'Ritual', 'Save',
       'School'],
      dtype='object')

In [163]:
link_list[0]

{'href': '/web/20190207025036/https://dnd5e.fandom.com/wiki/Acid_Splash',
 'title': 'Acid Splash',
 'html': <div class="mw-content-ltr mw-content-text" dir="ltr" id="mw-content-text" lang="en">
 <aside class="portable-infobox pi-background pi-europa pi-theme-wikia pi-layout-default">
 <h2 class="pi-item pi-item-spacing pi-title" data-source="name">Acid Splash</h2>
 <div class="pi-item pi-data pi-item-spacing pi-border-color" data-source="Level and Class">
 <div class="pi-data-value pi-font"><i><b>Conjuration cantrip</b></i><sup class="reference" id="cite_ref-spellDesc_0-0"><a href="#cite_note-spellDesc-0">[1]</a></sup></div>
 </div>
 <div class="pi-item pi-data pi-item-spacing pi-border-color" data-source="Casting Time">
 <h3 class="pi-data-label pi-secondary-font">Casting Time</h3>
 <div class="pi-data-value pi-font">1 action</div>
 </div>
 <div class="pi-item pi-data pi-item-spacing pi-border-color" data-source="Range">
 <h3 class="pi-data-label pi-secondary-font">Range</h3>
 <div cl

In [ ]:
# Also to grab:
 - Feats
 - Magic Items
 - Races
 - Subraces
 - Backgrounds
 - Monsters
 - 

In [ ]:
url_races = 'https://web.archive.org/web/20190208231105/https://dnd5e.fandom.com/wiki/Adventurers_League_Compliant_Races'
url_items = 'https://web.archive.org/web/20190208231105/https://dnd5e.fandom.com/wiki/Adventuring_Gear'
url_backgrounds = 'https://web.archive.org/web/20190208231102/https://dnd5e.fandom.com/wiki/Backgrounds'

In [205]:
soup_feats = BeautifulSoup(requests.get('https://web.archive.org/web/20190208231102/https://dnd5e.fandom.com/wiki/Feats').content)

In [206]:
soup_feats

<!DOCTYPE html>
<html class="" dir="ltr" lang="en">
<head><script src="//archive.org/includes/analytics.js?v=cf34f82" type="text/javascript"></script>
<script type="text/javascript">window.addEventListener('DOMContentLoaded',function(){var v=archive_analytics.values;v.service='wb';v.server_name='wwwb-app42.us.archive.org';v.server_ms=231;archive_analytics.send_pageview({});});</script><script charset="utf-8" src="/static/js/ait-client-rewrite.js?v=1553621510" type="text/javascript"></script>
<script type="text/javascript">
WB_wombat_Init("https://web.archive.org/web", "20190208231102", "dnd5e.fandom.com");
</script>
<script charset="utf-8" src="/static/js/wbhack.js?v=1553621510" type="text/javascript"></script>
<script type="text/javascript">
__wbhack.init('https://web.archive.org/web');
</script>
<link href="/static/css/banner-styles.css?v=1553621509" rel="stylesheet" type="text/css"/>
<link href="/static/css/iconochive.css?v=1553621509" rel="stylesheet" type="text/css"/>
<!-- End Way